In [ ]:
import re

import numpy as np
import requests
import ruptures as rpt  # our package
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# url = "https://raw.githubusercontent.com/BoiseState/CS121-Public/master/projects/p4-readerofbooks/etext/Moby-Dick-Broken.txt"
url = "https://sherlock-holm.es/stories/plain-text/houn.txt"

In [ ]:
r = requests.get(url)
# retrieving data from the URL using get method

In [ ]:
content = str(r.content)
chapter1 = content[content.find("CHAPTER I") : content.find("CHAPTER II")]
chapter2 = content[content.find("CHAPTER II") : content.find("CHAPTER III")]

# content = content[content.find("Original Transcriber's Notes:"):]
# chapter1 = content[content.find("CHAPTER 1. Loomings."):content.find("CHAPTER 2. The Carpet-Bag.")]
# chapter2 = content[content.find("CHAPTER 2. The Carpet-Bag."):content.find("CHAPTER 3. The Spouter-Inn.")]

In [ ]:
len(chapter1.split("."))

In [ ]:
chapter1_sentences = chapter1.split(".")
print(len(chapter1_sentences))

# Remove chapter section and chapter title, replace it by empy string
chapter1_sentences[0] = ""
chapter1_sentences[1] = ""

for i, sentence in enumerate(chapter1_sentences):
    chapter1_sentences[i] = re.sub("\\\\n", "", sentence)
    chapter1_sentences[i] = chapter1_sentences[i].lower()
    chapter1_sentences[i] = re.sub(
        r"""([!"#$%&\'()*+,-./:;<=>?@[\\\]^_`{|}~])""", " ", chapter1_sentences[i]
    )  # Non ponctuation
    chapter1_sentences[i] = re.sub(
        r"([^A-Za-z ])", " ", chapter1_sentences[i]
    )  # non letters
    chapter1_sentences[i] = re.sub(
        "\s+", " ", chapter1_sentences[i]
    )  # several spaces -> single space
    if len(chapter1_sentences[i]) < 3:
        chapter1_sentences[i] = ""

print(len(chapter1_sentences))

In [ ]:
chapter1_sentences

In [ ]:
chapter2_sentences = chapter2.split(".")
print(len(chapter2_sentences))

# Remove chapter section and chapter title, replace it by empy string
chapter2_sentences[0] = ""
chapter2_sentences[1] = ""

for i, sentence in enumerate(chapter2_sentences):
    chapter2_sentences[i] = re.sub("\\\\n", "", sentence)
    chapter2_sentences[i] = chapter2_sentences[i].lower()
    chapter2_sentences[i] = re.sub(
        r"""([!"#$%&\'()*+,-./:;<=>?@[\\\]^_`{|}~])""", " ", chapter2_sentences[i]
    )  # Non ponctuation
    chapter2_sentences[i] = re.sub(
        r"([^A-Za-z ])", " ", chapter2_sentences[i]
    )  # non letters
    chapter2_sentences[i] = re.sub(
        "\s+", " ", chapter2_sentences[i]
    )  # several spaces -> single space
    if len(chapter2_sentences[i]) < 3:
        chapter2_sentences[i] = ""

print(len(chapter2_sentences))

In [ ]:
chapter2_sentences

In [ ]:
chapter1_sentences = chapter1_sentences + chapter2_sentences

In [ ]:
len(chapter1_sentences)

In [ ]:
len(chapter1)

In [ ]:
chapter1 = chapter1 + chapter2

In [ ]:
from gensim.utils import simple_preprocess

chapter1_sentences = chapter1.split(".")
for idx, sentence in enumerate(chapter1_sentences):
    chapter1_sentences[idx] = simple_preprocess(sentence)

emb_dims = 50
word2vec = Word2Vec(sentences=chapter1_sentences, min_count=2, size=emb_dims, window=10)
word2vec.train(
    chapter1_sentences, total_examples=word2vec.corpus_count, epochs=word2vec.epochs
)  # train word vectors

In [ ]:
def get_embeddings_mean(word2vec_model, sentence):
    mean = np.zeros((1, emb_dims))
    vocab_keys = word2vec.wv.vocab.keys()
    in_vocab = [el for el in sentence if el in vocab_keys]
    if len(in_vocab) == 0:
        return np.full((1, emb_dims), np.nan)
    for el in in_vocab:
        mean = mean + word2vec.wv[el].reshape(1, -1)
    return mean / len(in_vocab)


get_embeddings_mean(word2vec, chapter1_sentences[7])

X = np.zeros((len(chapter1_sentences), emb_dims))
for idx, sentence in enumerate(chapter1_sentences):
    X[idx, :] = get_embeddings_mean(word2vec, sentence)

# np.c_[word2vec.wv['before'], word2vec.wv['visitor']].T.shape

In [ ]:
import matplotlib.pyplot as plt

plt.pcolor(X.T, cmap="RdBu")
plt.show()

In [ ]:
for idx, sentence in enumerate(chapter1_sentences):
    print(
        np.array(
            [word2vec.wv[el] for el in sentence if el in word2vec.wv.vocab.keys()]
        ).shape
    )
    chapter1_sentences[idx] = np.mean(
        np.array([word2vec.wv[el] for el in sentence if el in word2vec.wv.vocab.keys()])
    )

In [ ]:
chapter1_sentences

In [ ]:
# ? https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html
word2vec = Word2Vec(min_count=2)
word2vec.build_vocab(chapter1_sentences)  # prepare the model vocabulary
word2vec.train(
    chapter1_sentences, total_examples=word2vec.corpus_count, epochs=word2vec.epochs
)  # train word vectors

In [ ]:
from gensim.test.utils import common_texts

model = Word2Vec(sentences=common_texts, size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")
model.train([["hello", "world"]], total_examples=1, epochs=1)
model.wv.vocab
print(common_texts)

In [ ]:
word2vec.wv.vocab

In [ ]:
vectorizer = TfidfVectorizer(strip_accents="unicode")
X = vectorizer.fit_transform(chapter1_sentences)
print(X.shape)
print(vectorizer.get_feature_names())

In [ ]:
X.toarray().shape

In [ ]:
algo = rpt.Dynp(model="l2", min_size=1, jump=1).fit(X.toarray())
result = algo.predict(n_bkps=3)

In [ ]:
result

In [ ]:
def print_text_segments(txt, bkps):
    txt_sentences = txt.split(".")
    original = re.sub("\\\\n", "\n", txt).split(".")
    res_str = ""
    for i in range(len(txt_sentences)):
        if i in bkps:
            res_str = (
                res_str
                + "\n---------------------------------------------------------------------------\n"
            )
        res_str = res_str + original[i] + "."

    print(res_str)


print_text_segments(chapter1, result)